In [1]:
import numpy as np
import pandas as pd 
from sklearn.model_selection import train_test_split
import random 
import torch
import os 
from torch.utils.data import Dataset, DataLoader

In [ ]:
paths = []
labels = []
audio_files = ["Audio_Speech_Actors_01-24", "Audio_Song_Actors_01-24"]
drive_base_path = "/content/drive/MyDrive/MARS"

for main_file in audio_files:
    main_file_path = os.path.join(drive_base_path, main_file)
    for actor in os.listdir(main_file_path):
        actor_path = os.path.join(main_file_path, actor)
        for audio in os.listdir(actor_path):
            if audio.endswith(".wav"):
                path = os.path.join(actor_path, audio)
                paths.append(path)
                emotion = int(audio.split("-")[2])
                labels.append(emotion)


In [3]:
print("Length of paths: ",len(paths))
print("Length of labels: ",len(labels))
print("Maximum label: ",max(labels))
print("Minimum label: ",min(labels))
print("Data type of paths: ",type(paths[0]))
print("Data type of lables: ",type(labels[0]))


Length of paths:  2452
Length of labels:  2452
Maximum label:  8
Minimum label:  1
Data type of paths:  <class 'str'>
Data type of lables:  <class 'int'>


In [4]:
data = list(zip(paths, labels))
seed = random.Random(42)
seed.shuffle(data)
data[:5]

[('Audio_Song_Actors_01-24\\Actor_24\\03-02-03-02-02-01-24.wav', 3),
 ('Audio_Song_Actors_01-24\\Actor_02\\03-02-03-02-02-02-02.wav', 3),
 ('Audio_Speech_Actors_01-24\\Actor_14\\03-01-03-01-02-01-14.wav', 3),
 ('Audio_Speech_Actors_01-24\\Actor_17\\03-01-07-01-02-01-17.wav', 7),
 ('Audio_Song_Actors_01-24\\Actor_06\\03-02-06-01-02-01-06.wav', 6)]

In [5]:
df_combined = pd.DataFrame(data, columns=["path","label"])
df_combined.head()


,path,label
0,Audio_Song_Actors_01-24\Actor_24\03-02-03-02-0...,3
1,Audio_Song_Actors_01-24\Actor_02\03-02-03-02-0...,3
2,Audio_Speech_Actors_01-24\Actor_14\03-01-03-01...,3
3,Audio_Speech_Actors_01-24\Actor_17\03-01-07-01...,7
4,Audio_Song_Actors_01-24\Actor_06\03-02-06-01-0...,6


In [6]:
df_combined.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2452 entries, 0 to 2451
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   path    2452 non-null   object
 1   label   2452 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 38.4+ KB


In [7]:
df_combined.describe()

,label
count,2452.000000
mean,4.318108
std,2.020284
min,1.000000
25%,3.000000
50%,4.000000
75%,6.000000
max,8.000000


In [8]:
df_combined.loc[0]

path     Audio_Song_Actors_01-24\Actor_24\03-02-03-02-0...
label                                                    3
Name: 0, dtype: object

In [9]:
from pathlib import Path
df_combined["path"] = df_combined["path"].str.replace("\\", "/", regex=False)


In [10]:
df_combined.head()

,path,label
0,Audio_Song_Actors_01-24/Actor_24/03-02-03-02-0...,3
1,Audio_Song_Actors_01-24/Actor_02/03-02-03-02-0...,3
2,Audio_Speech_Actors_01-24/Actor_14/03-01-03-01...,3
3,Audio_Speech_Actors_01-24/Actor_17/03-01-07-01...,7
4,Audio_Song_Actors_01-24/Actor_06/03-02-06-01-0...,6


In [11]:
df_combined["label"] = df_combined["label"].apply(lambda x: x-1)
print("minimum label value: ", df_combined["label"].min())
print("maximum label value: ", df_combined["label"].max())

minimum label value:  0
maximum label value:  7


In [12]:
training_data, validation_data = train_test_split(df_combined, test_size=0.2, random_state=32)

In [13]:
training_data.head()

,path,label
2138,Audio_Speech_Actors_01-24/Actor_08/03-01-08-01...,7
623,Audio_Speech_Actors_01-24/Actor_05/03-01-01-01...,0
1942,Audio_Song_Actors_01-24/Actor_15/03-02-03-02-0...,2
1331,Audio_Speech_Actors_01-24/Actor_05/03-01-08-02...,7
401,Audio_Speech_Actors_01-24/Actor_20/03-01-02-02...,1


In [14]:
validation_data.head()

,path,label
1794,Audio_Speech_Actors_01-24/Actor_16/03-01-08-02...,7
1833,Audio_Speech_Actors_01-24/Actor_20/03-01-08-02...,7
1488,Audio_Song_Actors_01-24/Actor_17/03-02-05-02-0...,4
2267,Audio_Song_Actors_01-24/Actor_24/03-02-04-01-0...,3
290,Audio_Speech_Actors_01-24/Actor_13/03-01-04-02...,3


In [15]:
print("Length of Training Data: ", len(training_data))
print("Length of Validation Data: ", len(validation_data))

Length of Training Data:  1961
Length of Validation Data:  491


In [16]:
import soundfile as sf 
import torchaudio
import librosa
waveform, sr = sf.read(training_data.loc[0,"path"])
print((waveform.shape),sr)
waveform, sr = librosa.load(training_data.loc[0,"path"], sr=16000)
print(waveform.shape, sr)

(237037,) 48000
(79013,) 16000


In [17]:
max_duration = 0
longest_file = ""
for path in df_combined['path']:
    # full_path = os.path.join(base_dir, path)
    if os.path.isfile(path):
        waveform, sample_rate = librosa.load(path, sr=16000)
        duration = waveform.shape[0] / sample_rate
        if duration > max_duration:
            max_duration = duration
            longest_file = path
    else:
        print(f"one file path is not available {path}")
        break

print(f"Max duration: {max_duration:.8f} seconds")
print(f"sample rate: {sample_rate:.2f}")
print(f"Longest file: {longest_file}")

Max duration: 6.37306250 seconds
sample rate: 16000.00
Longest file: Audio_Song_Actors_01-24/Actor_22/03-02-02-02-02-01-22.wav


In [18]:
waveform, sr = librosa.load("Audio_Song_Actors_01-24/Actor_22/03-02-02-02-02-01-22.wav", sr =16000)
print(waveform.shape, sr)

(101969,) 16000


In [19]:
from collections import Counter
sample_rates = []
for full_path in df_combined['path']:
    # full_path = os.path.join(base_dir, path)
    if os.path.isfile(full_path):
        _, sr = librosa.load(full_path, sr=16000)
        sample_rates.append(sr)

# Count frequency of each sample rate
rate_counts = Counter(sample_rates)
print(rate_counts)

Counter({16000: 2452})


In [20]:
class AudioDataset(Dataset):
    def __init__(self, data, processor, max_length= int(6.37306250*16000)):
        self.data = data
        self.processor = processor
        self.max_length = max_length
    def __len__(self):
        return len(self.data)
    def __getitem__(self, index):
        audio_path = self.data.iloc[index]["path"]
        label = self.data.iloc[index]["label"]

        audio, sr = librosa.load(audio_path)
        audio = audio.squeeze()

        if len(audio) > self.max_length :
            audio = audio[:self.max_length]
            # print(f"found a audio file greater than max length : {audio_path} with an audio length {len(audio)}")
        else:
            audio = np.pad(audio, (0,int(self.max_length-len(audio))), "constant")
            
        inputs = self.processor(audio, sampling_rate=16000, return_tensors='pt', padding=True, truncate=True, max_length=self.max_length)
        input_values = inputs.input_values.squeeze()

        return {'input_values': input_values, 'labels': torch.tensor(label, dtype=torch.long)}

    

In [21]:
import torch
print(torch.__version__)


2.5.1+cu121


In [22]:
from transformers import Wav2Vec2Model, Wav2Vec2Processor, Wav2Vec2ForSequenceClassification, Wav2Vec2Config, Trainer, TrainingArguments
model = Wav2Vec2ForSequenceClassification.from_pretrained("facebook/wav2vec2-large", num_labels=8)
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base")

c:\Users\akhilesh\.conda\envs\akhilesh-env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\akhilesh\.conda\envs\akhilesh-env\Lib\site-packages\huggingface_hub\file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of Wav2Vec2ForSequenceClassification were not initialized from the model checkpoint at facebook/wav2vec2-large and are newly initialized: ['classifier.bias', 'classifier.weight', 'projector.bias', 'projector.weight', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1']
You should probably TRAIN this model on a down-strea

In [23]:
train_dataset = AudioDataset(training_data, processor)
validation_dataset = AudioDataset(validation_data, processor)

In [24]:
train_dataloader = DataLoader(train_dataset, batch_size=8, shuffle=True)
validation_dataloader = DataLoader(validation_dataset, batch_size=8, shuffle=False)

In [25]:
from transformers.utils import logging
logging.set_verbosity_info()

In [26]:
from transformers import __version__
print(__version__)


4.39.3


In [ ]:
from transformers import TrainingArguments
training_args = TrainingArguments(
    output_dir="./wav2vec2-large",
    overwrite_output_dir=True,
    eval_strategy="epoch",  # Changed from evaluation_strategy
    save_strategy="epoch",
    logging_strategy="epoch",
    metric_for_best_model="eval_accuracy",
    save_total_limit=10,
    logging_dir="./wav2vec2-large/logs",
    greater_is_better=True,
    gradient_accumulation_steps=2,
    learning_rate=2e-5,
    weight_decay=0.01,
    num_train_epochs=19,
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [28]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def get_metrics(pred):
    labels = pred.label_ids
    preds = np.argmax(pred.predictions, axis=1)
    accuracy = accuracy_score(labels, preds)
    precision, recall, fscore, _ = precision_recall_fscore_support(labels, preds, average="weighted")
    return {
        'accuracy':accuracy,
        'precision':precision,
        'recall':recall,
        'fscore':fscore
    }

In [29]:
import os
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

In [30]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=validation_dataset,
    compute_metrics=get_metrics
)


c:\Users\akhilesh\.conda\envs\akhilesh-env\Lib\site-packages\accelerate\accelerator.py:446: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [31]:
trainer.train()

***** Running training *****
  Num examples = 1,961
  Num Epochs = 19
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 2
  Total optimization steps = 2,337
  Number of trainable parameters = 315,693,448
  5%|▌         | 123/2337 [06:21<1:44:20,  2.83s/it]***** Running Evaluation *****
  Num examples = 491
  Batch size = 8


{'loss': 1.9064, 'grad_norm': 11.772726058959961, 'learning_rate': 1.894736842105263e-05, 'epoch': 1.0}


c:\Users\akhilesh\.conda\envs\akhilesh-env\Lib\site-packages\sklearn\metrics\_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
                                                    
  5%|▌         | 123/2337 [06:57<1:44:20,  2.83s/it]

{'eval_loss': 1.6625241041183472, 'eval_accuracy': 0.3727087576374745, 'eval_precision': 0.3084582023706751, 'eval_recall': 0.3727087576374745, 'eval_fscore': 0.2881233079036416, 'eval_runtime': 35.7512, 'eval_samples_per_second': 13.734, 'eval_steps_per_second': 1.734, 'epoch': 1.0}


Saving model checkpoint to ./wav2vec2-large\checkpoint-123
Configuration saved in ./wav2vec2-large\checkpoint-123\config.json
Model weights saved in ./wav2vec2-large\checkpoint-123\model.safetensors
Deleting older checkpoint [wav2vec2-large\checkpoint-246] due to args.save_total_limit
 11%|█         | 246/2337 [13:46<1:37:07,  2.79s/it] ***** Running Evaluation *****
  Num examples = 491
  Batch size = 8


...

                                                     
100%|██████████| 2337/2337 [3:24:43<00:00,  2.67s/it]Saving model checkpoint to ./wav2vec2-large\checkpoint-2337
Configuration saved in ./wav2vec2-large\checkpoint-2337\config.json


{'eval_loss': 0.7227576375007629, 'eval_accuracy': 0.8839103869653768, 'eval_precision': 0.8871974205525632, 'eval_recall': 0.8839103869653768, 'eval_fscore': 0.881463162315191, 'eval_runtime': 34.477, 'eval_samples_per_second': 14.241, 'eval_steps_per_second': 1.798, 'epoch': 19.0}


Model weights saved in ./wav2vec2-large\checkpoint-2337\model.safetensors


Training completed. Do not forget to share your model on huggingface.co/models =)


100%|██████████| 2337/2337 [3:24:55<00:00,  5.26s/it]

{'train_runtime': 12295.9256, 'train_samples_per_second': 3.03, 'train_steps_per_second': 0.19, 'train_loss': 0.4926736000821689, 'epoch': 19.0}


TrainOutput(global_step=2337, training_loss=0.4926736000821689, metrics={'train_runtime': 12295.9256, 'train_samples_per_second': 3.03, 'train_steps_per_second': 0.19, 'train_loss': 0.4926736000821689, 'epoch': 19.0})

In [ ]:
trainer.train(resume_from_checkpoint="Give the path")

means that you are instructing the Hugging Face Trainer to resume training from a previously saved checkpoint located at the specified path ("Give the path"). This allows you to continue training from where you left off, rather than starting over from scratch.